## CS585 HW5

### Group Members: Jinghao Ye, Yuhang Sun

In [1]:
import cv2
import numpy as np
from os import listdir
from os.path import isfile, join
import random
from scipy.optimize import linear_sum_assignment

thresh = 128
max_thresh = 255

### Define Track and Tracker

In [2]:
class Track:
    # it defines single objects detected in frames
    def __init__(self, position, trackID, remove_time):
        self.position = position
        self.id = trackID # id of each track object
        self.ori_remove_time = remove_time
        self.remove_time = remove_time
        
        # initialize Kalman filter
        self.kf = cv2.KalmanFilter(2, 2)
        self.kf.measurementMatrix = np.array([[1, 0], [0, 1]], np.float32)
        self.kf.transitionMatrix = np.array([[1, 0], [0, 1]], np.float32)
        self.kf.processNoiseCov = np.array([[1, 0],[0, 1]], np.float32) * 0.07
        self.kf.measurementNoiseCov = np.array([[1, 0],[0, 1]], np.float32) * 0.01
        self.trace = []
        self.color = (random.randint(0, 255), random.randint(0, 255), random.randint(0, 255))
        self.stop = False
    
    def drawPath(self, image):
        # draw the path of the object
        if len(self.trace) > 1:
            for i in range(1, len(self.trace)):
                pre = self.trace[i - 1]
                cur = self.trace[i]
                cv2.circle(image, tuple(pre), 1, self.color, 2)
                cv2.circle(image, tuple(cur), 1, self.color, 2)
                cv2.line(image, tuple(pre), tuple(cur), self.color, 1)
                
    def predict(self):
        # predict new position
        x, y = self.kf.predict()
        return [int(x), int(y)]
    
    def correct(self, position):
        # update the track's state
        if self.stop:
            return self.stop
        if position is None:
            self.remove_time -= 1
            if self.remove_time == 0:
                self.stop = True
        else:
            correct = self.kf.correct(np.float32(position))
            int_correct = np.array([int(correct[0]), int(correct[1])])
            self.trace.append(int_correct)
            self.position = int_correct
            self.remove_time = self.ori_remove_time
        
        return self.stop

In [3]:
class Tracker:
    # tracker updates tracks of objects
    def __init__(self, dist_thresh, remove_time):
        self.dist_thresh = dist_thresh
        self.remove_time = remove_time
        self.tracks = []
        self.trackID = 0
        
    def initialize(self, measurements):
        # initialize the tracker
        for i in range(len(measurements)):
                track = Track(measurements[i], self.trackID, self.remove_time)
                self.trackID += 1
                self.tracks.append(track)
                
    def update(self, measurements, image):
        tracked_items = []
        tracked_items_id = []
        # make prediction
        for item in self.tracks:
            if not item.stop:
                prediction = item.predict()
                tracked_items.append(prediction)
                tracked_items_id.append(item.id)
                

        # use Kalman Filter to predict new state
        # then use Hungarian Algorithm to match exist tracks and new points
        # calculate cost using sum of square distance between predicted states and measurements
        N = len(tracked_items)
        M = len(measurements)
        cost = np.zeros(shape=(N, M)) # cost matrix
        for i in range(N):
            for j in range(M):
                diff = self.tracks[i].position - measurements[j]
                distance = np.sqrt(diff[0] ** 2 + diff[1] ** 2)
                cost[i][j] = distance
        
        # then use Hungarian Algorithm to assign the correct measurements
        assignment = []
        for _ in range(N):
            assignment.append(-1)
        row_ind, col_ind = linear_sum_assignment(cost)
        for i in range(len(row_ind)):
            assignment[row_ind[i]] = col_ind[i]
        
        # Identify tracks with no assignment, if any
        for i in range(len(assignment)):
            if (assignment[i] != -1):
                # check for cost distance threshold.
                # If cost is very high then un_assign (delete) the track
                if(cost[i][assignment[i]] > self.dist_thresh):
                    assignment[i] = -1
        
        # update tracks
        for i in range(N):
            if assignment[i] != -1:
                measurement = measurements[assignment[i]]
            else:
                measurement = None

            item = self.tracks[tracked_items_id[i]]
            stop = item.correct(measurement)
            if not stop:
                item.drawPath(image)
        
        # new tracks
        un_assigned_measure = []
        for i in range(len(measurements)):
            if i not in assignment:
                un_assigned_measure.append(i)
        
        if(len(un_assigned_measure) != 0):
            for i in range(len(un_assigned_measure)):
                track = Track(measurements[un_assigned_measure[i]], self.trackID, self.remove_time)
                self.trackID += 1
                self.tracks.append(track)

### Bats Part

In [4]:
img_path = './CS585-BatImages/Gray'
# read images
files = [join(img_path,f) for f in listdir(img_path) if isfile(join(img_path,f))]

In [5]:
img = []
for i in range(len(files)):
    img.append(cv2.imread(files[i]))
    
print(len(img))
# shape of img: 1024 * 1024 * 3

151


In [6]:
def readFromTxt(files):
    result = []
    for i in range(len(files)):
        with open(files[i], 'r') as f:
            frame = []
            for line in f.readlines():
                line = line.strip('\n')
                line = line.split(',')
                line = np.array(list(map(int, line)))
                frame.append(line)
            result.append(frame)
    return result

In [7]:
# read localizations
local_path = './Localization'
files = [join(local_path, f) for f in listdir(local_path) if isfile(join(local_path, f))]
local = readFromTxt(files)

print(len(local))

151


In [8]:
# bats
img_indx = 1
dis_thresh = 80
timer = 200
tracker = Tracker(dis_thresh, timer)
tracker.initialize(local[0])
while img_indx < len(files):
    cur = img[img_indx]
    track_output = img[img_indx][:,:,:].copy()
        
    cur_local = local[img_indx]
    if len(cur_local) > 0:
        for i in range(len(cur_local)):
            cv2.circle(track_output, (cur_local[i][0], cur_local[i][1]), 1, (0,255,0), 4)
        
        tracker.update(cur_local, track_output)
        
        # Show in a window
        cv2.namedWindow("Track", cv2.WINDOW_AUTOSIZE)
        cv2.imshow("Track", track_output)
        if cv2.waitKey(100)&0xFF == 27:
            break
    
    
    img_indx += 1

cv2.destroyAllWindows()

### Cells Part

In [9]:
def threshold(x, img):
    # Convert into binary image using thresholding
    # Documentation for threshold: http://docs.opencv.org/modules/imgproc/doc/miscellaneous_transformations.html?highlight=threshold#threshold
    # Example of thresholding: http://docs.opencv.org/doc/tutorials/imgproc/threshold/threshold.html
#     thres_output = cv2.adaptiveThreshold(img,255,cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY,17,-5)

#     # Find contours
# 	# Documentation for finding contours: http://docs.opencv.org/modules/imgproc/doc/structural_analysis_and_shape_descriptors.html?highlight=findcontours#findcontours
#     contours, hierarchy = cv2.findContours(thres_output, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    _, thresh = cv2.threshold(blur, 40, 255, cv2.THRESH_BINARY)    
    kernel = np.ones((5,5))
    dilated = cv2.dilate(thresh, kernel)
    dilated = cv2.dilate(dilated, kernel)
    contours, _ = cv2.findContours(dilated, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    
    return contours, thresh

def detectors(frame, track_output):
    contours, thres_output = threshold(thresh, frame)

    centers = []#center of object
    radius_thresh = 1
    if (len(contours) > 0):
        for i in range(len(contours)):
            try:
                (x, y), radius = cv2.minEnclosingCircle(contours[i])
                centeroid = (int(x), int(y))
                radius = int(radius)
                if (radius > radius_thresh):
                    cv2.circle(track_output, centeroid, radius, (0, 255, 0), 2)
                    line = [x,y]
                    center = np.array(list(map(int, line)))

                    centers.append(center)
            except ZeroDivisionError:
                pass

            
    return centers

In [10]:
img_path = './CS585-Cells'
# read images
files = [join(img_path,f) for f in listdir(img_path) if isfile(join(img_path,f))]

img = []
for i in range(len(files)):
    img.append(cv2.imread(files[i]))
    
print(len(img))

112


In [11]:
# cell
img_indx = 1
dis_thresh = 80
timer = 200
tracker = Tracker(dis_thresh, timer)
# tracker.initialize(local[0])
init_Flag = True
ori_img = img[0]
while img_indx < len(img):
    cur = img[img_indx]

    if cur is None:
        img_indx += 1
        pass
    else:
        diff = cv2.absdiff(ori_img, cur)
        gray = cv2.cvtColor(diff, cv2.COLOR_BGR2GRAY)

    #Blur the image
    blur = cv2.blur(gray, (5, 5),0)
    track_output = img[img_indx][:,:,:].copy()

    cur_local = detectors(blur,track_output)
    if init_Flag:
        tracker.initialize(cur_local)
        init_Flag = False
    
    if len(cur_local) > 0:
        for i in range(len(cur_local)):
            cv2.circle(track_output, (cur_local[i][0], cur_local[i][1]), 1, (0,255,0), 4)
    
        tracker.update(cur_local, track_output)
        
        # Show in a window
        cv2.namedWindow("Track", cv2.WINDOW_AUTOSIZE)
        cv2.imshow("Track", track_output)
        #cv2.imshow("Contours", cur)
        if cv2.waitKey(100)&0xFF == 27:
            break
    
    
    img_indx += 1

cv2.destroyAllWindows()